In [1]:
import warnings
from pylab import rcParams
from tensorflow_core.python.keras.wrappers.scikit_learn import KerasClassifier
rcParams['figure.figsize'] = 10, 5
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
import csv
from sklearn.metrics import accuracy_score
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../logis_norm.csv')
### split data en X et Y
data1= data.copy()
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']

X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)

### Neural Network
## ANN Model
def create_model(optimizer):
    model = Sequential()
	#Layer 1
    model.add(Dense(20, activation='relu', input_dim=20))
    model.add(Dropout(0.3))
    #Layer 2
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.3))
    
    # output layer
    model.add(Dense(units=1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [4]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=50)
ANN = KerasClassifier(build_fn=create_model)

params={'optimizer' : ['adam', 'rmsprop'],
        'batch_size': [128, 256, 512, 800]}


##accuracy
grid_search_acc= GridSearchCV(estimator=ANN, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train, epochs=300, validation_data=(X_test, Y_test), callbacks=[early_stop])
y_predict= grid_search_acc.best_estimator_.predict(X_test)

Train on 71786 samples, validate on 30766 samples
Epoch 1/300
71786/71786 [==============================] - 2s 31us/sample - loss: 0.6945 - accuracy: 0.5337 - val_loss: 0.6780 - val_accuracy: 0.5886
Epoch 2/300
71786/71786 [==============================] - 2s 23us/sample - loss: 0.6756 - accuracy: 0.5789 - val_loss: 0.6584 - val_accuracy: 0.6127
Epoch 3/300
71786/71786 [==============================] - 2s 27us/sample - loss: 0.6638 - accuracy: 0.6106 - val_loss: 0.6457 - val_accuracy: 0.6242
Epoch 4/300
71786/71786 [==============================] - 2s 27us/sample - loss: 0.6531 - accuracy: 0.6276 - val_loss: 0.6252 - val_accuracy: 0.6633
Epoch 5/300
71786/71786 [==============================] - 2s 27us/sample - loss: 0.6433 - accuracy: 0.6411 - val_loss: 0.6105 - val_accuracy: 0.7002
Epoch 6/300
71786/71786 [==============================] - 2s 26us/sample - loss: 0.6359 - accuracy: 0.6489 - val_loss: 0.6026 - val_accuracy: 0.7125
Epoch 7/300
71786/71786 [=========================

In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'ANN', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")

----------
best parameters:  {'batch_size': 128, 'optimizer': 'adam'}
----------
[[10612  4747]
 [ 3662 11745]]
----------
              precision    recall  f1-score   support

           0       0.74      0.69      0.72     15359
           1       0.71      0.76      0.74     15407

    accuracy                           0.73     30766
   macro avg       0.73      0.73      0.73     30766
weighted avg       0.73      0.73      0.73     30766

----------
